<a href="https://colab.research.google.com/github/cavs1010/personal-projects/blob/main/GC_Real_Estate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Data Collection

The data will be collected from https://www.domain.com.au/rent/gold-coast-qld/ <br> I tried using www.realestate.com.au but there is not permission.

In [1]:
# Install the main libraries for web scrapping
! pip3 install requests
! pip3 install beautifulsoup4

In [2]:
#Import Main Libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import math

In [3]:
URL = 'https://www.domain.com.au/rent/gold-coast-qld/'
page = requests.get(URL)

In [4]:
soup = BeautifulSoup(page.content, 'html.parser')

In [5]:
#Number of web pages to read
numberOfWebsToRead = soup.find("h1", attrs={"class": "css-ekkwk0"}).text
numberOfWebsToRead = int(re.findall(r'\d+', numberOfWebsToRead)[0])
numberOfWebsToRead = numberOfWebsToRead /21
numberOfWebsToRead = math.floor(numberOfWebsToRead)

In [6]:
 #Arrays for the ammenities in the house
 mainAddressArray = []
 secondAddressArray = []
 postalCodeArray = []
 rentArray = []
 bedsArray = []
 bathsArray = []
 parkingArray = []
 buildingTypeArray = []

for i in range (1,7):
  URL = 'https://www.domain.com.au/rent/gold-coast-qld/?page='+str(i)
  page = requests.get(URL)
  soup = BeautifulSoup(page.content, 'html.parser')
  #Each element will cotain the info required for my work
  elements = soup.find_all("div", class_="css-1n74r2t")
  for element in elements:
    mainAddress = element.find("span", attrs={"data-testid": "address-line1"})
    secondAddress = element.find("span", attrs={"itemprop": "addressLocality"})
    postalCode = element.find("span", attrs={"itemprop": "postalCode"})
    rent = element.find("p", attrs={"data-testid": "listing-card-price"})
    ammenities = element.find_all("span", attrs={"class": "css-1rzse3v"})
    buildingType = element.find("span", attrs={"class": "css-693528"})
    if all(label is not None for label in [mainAddress, secondAddress, postalCode, rent, ammenities[0], ammenities[1], ammenities[2], buildingType]):
      mainAddressArray.append(mainAddress.text)
      secondAddressArray.append(secondAddress.text)
      postalCodeArray.append(postalCode.text)
      rentArray.append(rent.text)
      bedsArray.append(ammenities[0].text)
      bathsArray.append(ammenities[1].text)
      parkingArray.append(ammenities[2].text)
      buildingTypeArray.append(buildingType.text)
    else:
      print('It was not possible to add an element in page '+str(i)+" becasue it's missing some values")

It was not possible to add an element in page 3 becasue it's missing some values


## Final data gathered:

In [7]:
dictionary = {'Main Address': mainAddressArray, 
              'Second Address': secondAddressArray, 
              'Postal Code': postalCodeArray, 
              'Rent':rentArray, 
              'Bedrooms': bedsArray, 
              'Bathrooms': bathsArray,
              'Parking spaces': parkingArray,
              'Building Type': buildingTypeArray}
df = pd.DataFrame(dictionary)
df

,Main Address,Second Address,Postal Code,Rent,Bedrooms,Bathrooms,Parking spaces,Building Type
0,"6 Cambridge Circuit,",PIMPAMA,4209,$490 Weekly,4 .css-9fxapx{position:absolute;width:1px;heig...,2 Baths,2 Parking,House
1,"16 Lanai Drive,",BURLEIGH WATERS,4220,"$1,150 per week",5 Beds,2 Baths,2 Parking,House
2,"2/22 Britannia Avenue,",BROADBEACH,4218,$700 per week,2 Beds,2 Baths,1 Parking,Apartment / Unit / Flat
3,"12 Kosciuszko Circuit,",PIMPAMA,4209,$650,5 Beds,2 Baths,2 Parking,House
4,"9 Purlingbrook Street,",UPPER COOMERA,4209,$470,3 Beds,1 Bath,2 Parking,House
...,...,...,...,...,...,...,...,...
92,"53 Amalfi Drive,",ISLE OF CAPRI,4217,$1350 per week,5 Beds,3 Baths,2 Parking,House
93,"20 Brakes Crescent,",MIAMI,4220,"$2,600 per week",5 Beds,3 Baths,2 Parking,House
94,"19 Norfolk Avenue,",SURFERS PARADISE,4217,$600 per week,3 Beds,2 Baths,2 Parking,House
95,"18/6 Tonga Place,",PARKWOOD,4214,$475,1 Bed,1 Bath,1 Parking,House


# 2. Data Cleaning

In [37]:
dfCopy = df.copy()

In [38]:
#Function that transform the columns[Bedrooms, Bathrooms and Parking Spaces] to integers
#columnName : Name of the column to transform into an integer.
def toInteger (columnName):
  for ind in dfCopy[columnName].index:
    value = dfCopy[columnName][ind][0]
    if (value == '−'):
      value = 0
    else:
      value = int(value)
    dfCopy[columnName][ind] = value

In [39]:
toInteger('Bathrooms')
toInteger('Bedrooms')
toInteger('Parking spaces')
dfCopy

,Main Address,Second Address,Postal Code,Rent,Bedrooms,Bathrooms,Parking spaces,Building Type
0,"6 Cambridge Circuit,",PIMPAMA,4209,$490 Weekly,4,2,2,House
1,"16 Lanai Drive,",BURLEIGH WATERS,4220,"$1,150 per week",5,2,2,House
2,"2/22 Britannia Avenue,",BROADBEACH,4218,$700 per week,2,2,1,Apartment / Unit / Flat
3,"12 Kosciuszko Circuit,",PIMPAMA,4209,$650,5,2,2,House
4,"9 Purlingbrook Street,",UPPER COOMERA,4209,$470,3,1,2,House
...,...,...,...,...,...,...,...,...
92,"53 Amalfi Drive,",ISLE OF CAPRI,4217,$1350 per week,5,3,2,House
93,"20 Brakes Crescent,",MIAMI,4220,"$2,600 per week",5,3,2,House
94,"19 Norfolk Avenue,",SURFERS PARADISE,4217,$600 per week,3,2,2,House
95,"18/6 Tonga Place,",PARKWOOD,4214,$475,1,1,1,House


## Transform the rent into a float:

In [40]:

rentIntArray = []
for ind in dfCopy['Rent'].index:
  #Extract the numbers from the string
  lista = re.findall(r'\d+', dfCopy['Rent'][ind])
  finalRent = ''.join(lista)
  #Eliminate the row if the number is ''
  if (finalRent == ''):
    dfCopy.drop([ind], inplace=True)
    print ('This is the problem index: '+str(ind)+', so it was eliminated' )
  #Otherwise, it add the number to a list
  else:
    dfCopy['Rent'][ind] = float(finalRent)
    rentIntArray.append(float(finalRent))

In [41]:
dfCopy.head()

,Main Address,Second Address,Postal Code,Rent,Bedrooms,Bathrooms,Parking spaces,Building Type
0,"6 Cambridge Circuit,",PIMPAMA,4209,490,4,2,2,House
1,"16 Lanai Drive,",BURLEIGH WATERS,4220,1150,5,2,2,House
2,"2/22 Britannia Avenue,",BROADBEACH,4218,700,2,2,1,Apartment / Unit / Flat
3,"12 Kosciuszko Circuit,",PIMPAMA,4209,650,5,2,2,House
4,"9 Purlingbrook Street,",UPPER COOMERA,4209,470,3,1,2,House


## Get the latitude and Longitude

In [42]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="my_user_agent")


In [47]:
housesNotFound = []
for index, row in dfCopy.iterrows():
  if '/' in row['Main Address']:
    location =  geolocator.geocode(row['Main Address'].split('/',1)[1]+' '+row['Second Address'] + ', QLD, Australia') 
  else:
    location = geolocator.geocode(row['Main Address']+' '+row['Second Address']  + ', QLD, Australia') 
  if location == None:
    housesNotFound.append( row['Main Address']+row['Second Address'])
    dfCopy.drop([index], inplace=True)
  else:
    dfCopy.at[index,'Latitude'] = location.latitude
    dfCopy.at[index,'Longitude'] = location.longitude


print('These houses were not found:'+ str(housesNotFound))

These houses were not found:[]


In [48]:
housesNotFound

[]

In [49]:
dfCopy

,Main Address,Second Address,Postal Code,Rent,Bedrooms,Bathrooms,Parking spaces,Building Type,Latitude,Longitude
0,"6 Cambridge Circuit,",PIMPAMA,4209,490,4,2,2,House,-27.835562,153.315019
1,"16 Lanai Drive,",BURLEIGH WATERS,4220,1150,5,2,2,House,-28.095786,153.434559
2,"2/22 Britannia Avenue,",BROADBEACH,4218,700,2,2,1,Apartment / Unit / Flat,-28.023812,153.430991
3,"12 Kosciuszko Circuit,",PIMPAMA,4209,650,5,2,2,House,-27.816138,153.299638
4,"9 Purlingbrook Street,",UPPER COOMERA,4209,470,3,1,2,House,-27.864833,153.287389
...,...,...,...,...,...,...,...,...,...,...
91,"65 River Crescent,",BROADBEACH WATERS,4218,1600,4,2,3,House,-28.026044,153.398184
93,"20 Brakes Crescent,",MIAMI,4220,2600,5,3,2,House,-28.071295,153.444774
94,"19 Norfolk Avenue,",SURFERS PARADISE,4217,600,3,2,2,House,-27.994045,153.426043
95,"18/6 Tonga Place,",PARKWOOD,4214,475,1,1,1,House,-27.956883,153.380885


### 3. Data Analysis

In [ ]:
! pip install folium

In [ ]:
import folium

In [ ]:
goldCoastMap = folium.Map(location = [-28.0167, 153.4000],  
                          zoom_start = 10,
                          tiles="CartoDB positron")

In [ ]:
goldCoastMap

In [ ]:
folium.Circle(
    radius=100,
    location=[45.5244, -122.6699],
    popup="The Waterfront",
    color="crimson",
    fill=False,
).add_to(goldCoastMap)
goldCoastMap